<html>
<body style="background-color:powderblue;">
<p><i>Author: Shaheer Mansoor</i></p>


## 1. Spark Transformations & Actions

In this section we will run some transformations and actions


In [1]:
print sc.applicationId

application_1554311896068_9231


In [2]:
df = spark.table("ddp_cvm.uc5_ads")

In [3]:
import time

start = time.time()

#spark.table("ddp_cvm.uc5_ads").printSchema()
df = spark.table("ddp_cvm.uc5_ads")
df = df.select("mortgage_lapsed_mortages")

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Time Taken {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

Time Taken 00:00:01.43


In [4]:
#df.explain()
start = time.time()

print df.count()

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Time Taken {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

2088799
Time Taken 00:00:18.78


## 2. Spark Joins

In this section we join two tables in Spark

We will take the customer table and transaction for 201801 onwards and calculate the Sum for each customer

In [5]:
import time

start = time.time()

#spark.table("ddp_cvm.uc5_pop_distinct").printSchema()
df_cust = spark.table("ddp_central.src_sdw_part_party").filter("validtodate = '2999-12-31'").\
                select('partygenid').distinct()
print df_cust.count()
#12,551,554 rows

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Time Taken {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

12551554
Time Taken 00:00:12.85


In [6]:
start = time.time()

df_trx = spark.table("ddp_central.depo_transaction_t05fa")\
        .select('primaryaccountholder', 'transactionmonth', 'amount')\
        .filter("transactionmonth = 201801")
print df_trx.count()
#223,568,890 rows

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Time Taken {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

223568890
Time Taken 00:00:11.63


### 2.1 Explain the query plan



In [7]:
from pyspark.sql.functions import *

df_joined = df_trx.join(df_cust, df_trx.primaryaccountholder == df_cust.partygenid, how = 'inner')

df_sums = df_joined.groupby('partygenid').agg(sum("amount"))
df_sums.explain()

== Physical Plan ==
*HashAggregate(keys=[partygenid#3062], functions=[sum(amount#3174)])
+- *HashAggregate(keys=[partygenid#3062], functions=[partial_sum(amount#3174)])
   +- *Project [amount#3174, partygenid#3062]
      +- *SortMergeJoin [primaryaccountholder#3169], [partygenid#3062], Inner
         :- *Sort [primaryaccountholder#3169 ASC NULLS FIRST], false, 0
         :  +- Exchange hashpartitioning(primaryaccountholder#3169, 200)
         :     +- *Filter isnotnull(primaryaccountholder#3169)
         :        +- HiveTableScan [primaryaccountholder#3169, amount#3174], HiveTableRelation `ddp_central`.`depo_transaction_t05fa`, org.apache.hadoop.hive.ql.io.orc.OrcSerde, [agreementgenid#3163, transactiondate#3164, transactiongenid#3165, transtypecd#3166, servicetime#3167, bankid#3168, primaryaccountholder#3169, productgenid#3170, performedbyorggenid#3171, channeltypecd#3172, channelgenid#3173, amount#3174, originalamount#3175, originalcurrencycd#3176, accountingtypecd#3177, secpartygeni

### 2.2 Run the join 
(Dont Run!)

In [8]:
start = time.time()

df_joined = df_trx.join(df_cust, df_trx.primaryaccountholder == df_cust.partygenid, 'inner')

print df_joined.count()
#223,568,890 rows

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Time Taken {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

223568890
Time Taken 00:00:31.00


In [9]:
from pyspark.sql.functions import broadcast, sum

start = time.time()

df_sums = df_joined.groupby('partygenid').agg(sum("amount"))
print df_sums.count()
print df_sums.show(10)
#4,633,019 rows

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Time Taken {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

4633019
+--------------------+------------+
|          partygenid| sum(amount)|
+--------------------+------------+
|[00 3E 86 81 9C 6...|   892.50000|
|[00 4D 5A 62 70 A...| 12965.62000|
|[00 8D 6C 37 46 7...| 11908.67000|
|[00 A9 17 96 EB C...| 12300.18000|
|[00 B5 89 00 1E A...| -1997.81000|
|[00 BC 11 29 3A 1...|  1673.05000|
|[00 EB 4C 4A 0B 0...|-59059.97000|
|[01 92 84 C7 25 1...|-64634.91000|
|[01 B0 F1 4B 44 A...|   420.57000|
|[01 E6 CF D5 39 7...|-10065.58000|
+--------------------+------------+
only showing top 10 rows

None
Time Taken 00:00:31.06


### 2.2 Run the join again, this time we will cache our result

In [10]:
start = time.time()

df_joined = df_trx.join(df_cust, df_trx.primaryaccountholder == df_cust.partygenid, 'inner').cache()
print df_joined.count()

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Time Taken {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

223568890
Time Taken 00:00:20.27


In [11]:
from pyspark.sql.functions import broadcast, sum

start = time.time()

df_sums = df_joined.groupby('partygenid').agg(sum("amount"))
print df_sums.count()
print df_sums.show(10)

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Time Taken {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

4633019
+--------------------+------------+
|          partygenid| sum(amount)|
+--------------------+------------+
|[00 3E 86 81 9C 6...|   892.50000|
|[00 4D 5A 62 70 A...| 12965.62000|
|[00 8D 6C 37 46 7...| 11908.67000|
|[00 A9 17 96 EB C...| 12300.18000|
|[00 B5 89 00 1E A...| -1997.81000|
|[00 BC 11 29 3A 1...|  1673.05000|
|[00 EB 4C 4A 0B 0...|-59059.97000|
|[01 92 84 C7 25 1...|-64634.91000|
|[01 B0 F1 4B 44 A...|   420.57000|
|[01 E6 CF D5 39 7...|-10065.58000|
+--------------------+------------+
only showing top 10 rows

None
Time Taken 00:00:01.41


### 2.2.1 Cache the tables after reading them into Spark

In [12]:
df_joined.unpersist(True)

import pyspark

start = time.time()

#spark.table("ddp_cvm.uc5_pop_distinct").printSchema()
df_cust_cached = spark.table("ddp_central.src_sdw_part_party").filter("validtodate = '2999-12-31'").\
                       select('partygenid').distinct().cache()
print df_cust_cached.count()
#12,551,554 rows

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Time Taken {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

12551554
Time Taken 00:00:06.28


In [13]:
start = time.time()

#spark.table("ddp_cvm.uc5_pop_distinct").printSchema()
df_trx_cached = spark.table("ddp_central.depo_transaction_t05fa")\
        .selectExpr('primaryaccountholder as partygenid', 'amount')\
        .filter("transactionmonth = 201801")
        
df_trx_cached.cache()
#df_trx_cached.rdd.persist(pyspark.StorageLevel.MEMORY_ONLY_2)
print df_trx_cached.rdd.getStorageLevel()
#Action
print df_trx_cached.count()
#223,568,890 rows

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Time Taken {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

Serialized 1x Replicated
223568890
Time Taken 00:00:06.70


In [14]:
from pyspark.sql.functions import broadcast, sum

start = time.time()

df_joined_c = df_trx_cached.join(df_cust_cached, ['partygenid'], 'inner')
df_sums = df_joined_c.groupby('partygenid').agg(sum("amount"))
#Action
print df_sums.count()
print df_sums.show(10)
#4,633,019 rows

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Time Taken {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

4633019
+--------------------+------------+
|          partygenid| sum(amount)|
+--------------------+------------+
|[00 3E 86 81 9C 6...|   892.50000|
|[00 4D 5A 62 70 A...| 12965.62000|
|[00 8D 6C 37 46 7...| 11908.67000|
|[00 A9 17 96 EB C...| 12300.18000|
|[00 B5 89 00 1E A...| -1997.81000|
|[00 BC 11 29 3A 1...|  1673.05000|
|[00 EB 4C 4A 0B 0...|-59059.97000|
|[01 92 84 C7 25 1...|-64634.91000|
|[01 B0 F1 4B 44 A...|   420.57000|
|[01 E6 CF D5 39 7...|-10065.58000|
+--------------------+------------+
only showing top 10 rows

None
Time Taken 00:00:14.80


In [ ]:
#partitions = df_trx_cached.rdd.getNumPartitions()
#print("Partitions: {0:,}".format(partitions ))

### 2.3 Run the join again, after repartitioning

In [18]:
#from pyspark.sql.functions import broadcast

start = time.time()

df_joined_c = df_trx_cached.join(df_cust_cached, ['partygenid'], 'inner')
df_sums = df_joined_c.groupby('partygenid').agg(sum("amount"))

#Action
print df_sums.count()
#4,633,019 rows

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Time Taken {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

5032428
Time Taken 00:01:10.59


### 2.3 Adjust Partitioning to improve Join Times

In [17]:
#100, 500, 1000, 1500, 2000

df_trx_cached = spark.table("ddp_central.depo_transaction_t05fa")\
        .selectExpr('primaryaccountholder as partygenid', 'amount')\
        .filter("transactionmonth > 201809")       
df_trx_cached.cache()

df_trx_cached = df_trx_cached.repartition(45)

partitions = df_trx_cached.rdd.getNumPartitions()
print("Partitions: {0:,}".format(partitions ))
print("Records: {0:,}".format(df_trx_cached.count()))

df_cust_cached = df_cust_cached.repartition(45)

partitions = df_cust_cached.rdd.getNumPartitions()
print("Partitions: {0:,}".format(partitions ))
print("Records: {0:,}".format(df_cust_cached.count()))

#sc._conf.getAll()

Partitions: 45
Records: 1,418,059,301
Partitions: 45
Records: 12,551,554


### 2.4 View your Spark configuration 

In [19]:
sc._conf.getAll()

[(u'spark.eventLog.enabled', u'true'),
 (u'spark.dynamicAllocation.minExecutors', u'1'),
 (u'spark.driver.memory', u'2G'),
 (u'spark.history.kerberos.principal', u'spark-SWEDBANKSEHA2@FSPA.MYNTET.SE'),
 (u'spark.history.ui.port', u'18081'),
 (u'spark.driver.extraLibraryPath',
  u'/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 (u'spark.dynamicAllocation.maxExecutors', u'15'),
 (u'spark.ui.killEnabled', u'true'),
 (u'spark.history.kerberos.enabled', u'true'),
 (u'spark.dynamicAllocation.executorIdleTimeout', u'60'),
 (u'spark.executor.extraLibraryPath',
  u'/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 (u'spark.yarn.historyServer.address', u'sb-hdp-m3.fspa.myntet.se:18081'),
 (u'spark.history.provider',
  u'org.apache.spark.deploy.history.FsHistoryProvider'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.dynamicAllocation.initialExecutors', u'1'),
 (u'spark.execu